In [34]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin


Funciones auxiliares

In [35]:
def leer_csv(ruta):
    return pd.read_csv(ruta, dtype={
    'tipodepropiedad':'category', 'ciudad':'category',\
    'provincia':'category'}, parse_dates=[16])

# funcion para setear los superficies
def set_metros(row):
    total = row.metrostotales
    covered = row.metroscubiertos
    if np.isnan(total):
        row.metrostotales = covered
        return row
    if np.isnan(covered):
        row.metroscubiertos = total
        return row
    return row

# Preprocessing 

Imputing Functions

In [36]:
# define numerical imputer
num_imputer = SimpleImputer(strategy='mean')

class SeriesImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        If the Series is of dtype Category, then impute with the most frequent object.
        """
    def fit(self, X, y=None):
        self.fill = X.value_counts().index[0]
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

# define categorical imputer
cat_imputer = SeriesImputer()

def fill_valores(df, zeros, num_cols, cat_cols):
    if zeros:
        df[zeros] = df[zeros].fillna(value=0)
    if num_cols:
        df[num_cols] = num_imputer.fit_transform(df[num_cols])
    if cat_cols:
        for col in cat_cols:
            df[col] = cat_imputer.fit_transform(df[col])
    return df

Encoding functions

In [37]:
def one_hot_encoder(df, columnas):
    for col in columnas:
        df = pd.get_dummies(df, prefix = ['cat'], columns = [col])
    return df

Procesamiento de valores nulos

In [38]:
def processing_nulos(df, zeros_cols, num_cols, cat_cols, eliminar_sin_tipo=False):
    
    # Metros cubiertos y totales
    
    df[['metrostotales', 'metroscubiertos']] = \
    df[['metrostotales', 'metroscubiertos']].apply(set_metros, axis = 1)
    
    # Imputando otras columnas
    
    df = fill_valores(df, zeros_cols, num_cols, cat_cols)
    
    return df

Procesamiento de Fechas

In [39]:
def processing_fechas(df, anio, mes, dia):
    df['fecha'] = pd.to_datetime(df['fecha'])
        
    if anio:
        df['anio'] = df['fecha'].dt.year
    if mes:
        df['mes'] = df['fecha'].dt.month
    if dia:
        df['dia'] = df['fecha'].dt.day
    return df

Función suma_columnas

In [40]:
def suma_columnas(df, cols):
    nombre = ''
    for col in cols:
        nombre = nombre + col
    df['suma_'+nombre] = sum([df[col] for col in cols])
    return df

_____
### Funcion de preprocessing

In [41]:
def cambiar_valores(x):
    i = 0
    if x.name == 'precioporm2':
        i+=1
        return i
    return x

def encoder_idzona_ordinal(df_train, df_test): 
    
    df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']

    precio_m2_por_idzona = df_train.groupby('idzona').agg({'precioporm2' : 'mean'})
    precio_m2_por_idzona.reset_index(inplace = True)
    precio_m2_por_idzona = precio_m2_por_idzona.sort_values(by = 'precioporm2')

    precio_m2_por_idzona = precio_m2_por_idzona.apply(cambiar_valores)
    precio_m2_por_idzona['precioporm2'] = precio_m2_por_idzona['precioporm2'].cumsum()

    orden_idzona = precio_m2_por_idzona.set_index('idzona').to_dict()
    orden_idzona = orden_idzona['precioporm2']

    df_train['idzona_ordinal'] = cat_imputer.fit_transform(df_train.idzona.map(orden_idzona))
    df_test['idzona_ordinal'] = cat_imputer.fit_transform(df_test.idzona.map(orden_idzona))
    
    return df_train, df_test


In [42]:
def encoder_ciudad_ordinal(df_train, df_test): 
    
    df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']

    precio_m2_por_ciudad = df_train.groupby('ciudad').agg({'precioporm2' : 'mean'})
    precio_m2_por_ciudad.reset_index(inplace = True)
    precio_m2_por_ciudad = precio_m2_por_ciudad.sort_values(by = 'precioporm2')

    precio_m2_por_ciudad = precio_m2_por_ciudad.apply(cambiar_valores)
    precio_m2_por_ciudad['precioporm2'] = precio_m2_por_ciudad['precioporm2'].cumsum()

    orden_ciudad = precio_m2_por_ciudad.set_index('ciudad').to_dict()
    orden_ciudad = orden_ciudad['precioporm2']

    df_train['ciudad_ordinal'] = cat_imputer.fit_transform(df_train.ciudad.map(orden_ciudad))
    df_test['ciudad_ordinal'] = cat_imputer.fit_transform(df_test.ciudad.map(orden_ciudad))
    
    return df_train, df_test

In [43]:
def encoder_provincia_ordinal(df_train, df_test): 
    
    df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']

    precio_m2_por_provincia = df_train.groupby('provincia').agg({'precioporm2' : 'mean'})
    precio_m2_por_provincia.reset_index(inplace = True)
    precio_m2_por_provincia = precio_m2_por_provincia.sort_values(by = 'precioporm2')

    precio_m2_por_provincia = precio_m2_por_provincia.apply(cambiar_valores)
    precio_m2_por_provincia['precioporm2'] = precio_m2_por_provincia['precioporm2'].cumsum()

    orden_provincia = precio_m2_por_provincia.set_index('provincia').to_dict()
    orden_provincia = orden_provincia['precioporm2']

    df_train['provincia_ordinal'] = cat_imputer.fit_transform(df_train.provincia.map(orden_provincia))
    df_test['provincia_ordinal'] = cat_imputer.fit_transform(df_test.provincia.map(orden_provincia))
    
    return df_train, df_test

In [44]:
def encoder_ciudad_por_precio_m2(df_train, df_test):
    #filtrando los precios
    #hay que eliminar los nulls primero
    df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']
    df_train_ciudad = df_train.groupby('ciudad').agg({'precioporm2':'mean'}).sort_values('precioporm2').reset_index()
    #df_train_ciudad['precioporm2'] = np.log(df_train_ciudad['precioporm2'])
    ciudad_log_precio = df_train_ciudad.set_index('ciudad').to_dict()['precioporm2']

    df_test['ciudad_promedio_m2'] = cat_imputer.fit_transform(df_test.ciudad.map(ciudad_log_precio))
    
    df_train['ciudad_promedio_m2'] = cat_imputer.fit_transform(df_train.ciudad.map(ciudad_log_precio))
    
    return df_train, df_test

In [45]:
def encoder_idzona_por_precio_m2(df_train, df_test):
    #filtrando los precios
    #hay que eliminar los nulls primero
    df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']
    df_train_idzona = df_train.groupby('idzona').agg({'precioporm2':'mean'}).sort_values('precioporm2').reset_index()
    #df_train_idzona['precioporm2'] = np.log(df_train_idzona['precioporm2'])
    idzona_log_precio = df_train_idzona.set_index('idzona').to_dict()['precioporm2']

    df_test['idzona_promedio_m2'] = cat_imputer.fit_transform(df_test.idzona.map(idzona_log_precio))
    
    df_train['idzona_promedio_m2'] = cat_imputer.fit_transform(df_train.idzona.map(idzona_log_precio))
    
    return df_train, df_test

In [46]:
def encoder_posicion_por_precio_m2(df_train, df_test):
    #modificar las lat y lon en una colunma
    df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']
    #modificar lat y lon
    df_train['lat'] = df_train['lat'].apply(lambda x: round(x,2))    
    df_train['lng'] = df_train['lng'].apply(lambda x: round(x,2))
    df_test['lat'] = df_test['lat'].apply(lambda x: round(x,2))    
    df_test['lng'] = df_test['lng'].apply(lambda x: round(x,2))
    
    #unir lat y lon como posicion
    df_train['posicion'] = df_train['lat'].map(str) + df_train['lng'].map(str)
    df_test['posicion'] = df_test['lat'].map(str) + df_test['lng'].map(str)
    
    df_train_posicion = df_train.groupby('posicion').agg({'precioporm2':'mean'}).sort_values('precioporm2').reset_index()
    df_train_posicion = df_train_posicion[df_train_posicion.posicion != 'nannan']
    #df_train_idzona['precioporm2'] = np.log(df_train_idzona['precioporm2'])
    posicion_log_precio = df_train_posicion.set_index('posicion').to_dict()['precioporm2']

    df_test['posicion_promedio_m2'] = df_test.posicion.map(posicion_log_precio)
    
    df_train['posicion_promedio_m2'] = df_train.posicion.map(posicion_log_precio)
    
    return df_train, df_test

In [47]:
def preprocessing(guardar_csv=False):
    df_train = leer_csv('data/train.csv')
    df_test  = leer_csv('data/test.csv')
    
    df_train = df_train[(df_train['tipodepropiedad'] != 'Hospedaje')&\
                        (df_train['tipodepropiedad'] != 'Garage')]
    
    ## Fechas
    
    df_train = processing_fechas(df_train, True, True, True)
    df_test = processing_fechas(df_test, True, True, True)
    
    ## Nulos
    
    zeros_cols_train = None
    num_cols_train = ['antiguedad', 'habitaciones', 'banos', 'garages']
    cat_cols_train = ['ciudad', 'tipodepropiedad']
    
    zeros_cols_test = None
    num_cols_test = ['antiguedad', 'habitaciones', 'banos', 'garages']
    cat_cols_test = ['ciudad', 'tipodepropiedad']
    
    df_train = processing_nulos(df_train, zeros_cols_train, num_cols_train, cat_cols_train, eliminar_sin_tipo=True)
    df_test = processing_nulos(df_test, zeros_cols_test, num_cols_test, cat_cols_test)
    
    ## Encoding de variables categóricas
    
    # One Hot encoding para 'tipo de propiedad'
    df_train['tipodepropiedad'].cat.remove_unused_categories()
    df_train = one_hot_encoder(df_train, ['tipodepropiedad'])
    df_test = one_hot_encoder(df_test, ['tipodepropiedad'])
    
    # Encoding por precio promedio por m2 de las ciudades
    
    df_train, df_test = encoder_ciudad_por_precio_m2(df_train, df_test)
    df_train, df_test = encoder_idzona_por_precio_m2(df_train, df_test)
    df_train, df_test = encoder_posicion_por_precio_m2(df_train, df_test)
    
    # Encoding ordinal de idzona y ciudad
    
    df_train, df_test = encoder_idzona_ordinal(df_train, df_test)
    df_train, df_test = encoder_ciudad_ordinal(df_train, df_test)   
    df_train, df_test = encoder_provincia_ordinal(df_train, df_test)   
    
    
    ## Otras Features
    
    
    # 'cant_extras' = 'gimnasio'+'usosmultiples'+'piscina'
    extras = ['gimnasio', 'usosmultiples', 'piscina']
    df_train = suma_columnas(df_train, extras)
    df_test = suma_columnas(df_test, extras)
    
    # 'cant_cercanos' = 'escuelascercanas'+'centroscomercialescercanos'
    cercanos = ['escuelascercanas', 'centroscomercialescercanos']
    df_train = suma_columnas(df_train, cercanos)
    df_test = suma_columnas(df_test, cercanos)
    
    
    ## Eliminamos columnas no utilizadas
    df_train.drop(columns=['fecha', 'id', 'titulo', 'descripcion','direccion',\
                           'lat', 'lng', 'posicion','provincia','ciudad','gimnasio','usosmultiples',\
                           'escuelascercanas','centroscomercialescercanos',\
                           'precioporm2', 'cat_Garage', 'cat_Hospedaje'], inplace=True)
    df_test.drop(columns=['fecha', 'id', 'titulo', 'descripcion', 'direccion',\
                          'lat', 'lng','posicion','provincia', 'ciudad','gimnasio','usosmultiples',\
                          'escuelascercanas','centroscomercialescercanos',\
                          ], inplace=True)
    
    ## Agregamos columna con log(precio)
    df_train['log_precio'] = np.log(df_train['precio'])
    
    if guardar_csv:
        df_train.to_csv('data/train_preproc.csv', index = False)
        df_test.to_csv('data/test_preproc.csv', index = False)
    return df_train, df_test

In [48]:
df_train, df_test = preprocessing(guardar_csv=True)

In [49]:
# tiene 2 columnas más: precio y log_precio
df_train.columns.nunique()

43

In [50]:
df_test.columns.nunique()

41

In [58]:
df_test.isnull().sum()

antiguedad                                             0
habitaciones                                           0
garages                                                0
banos                                                  0
metroscubiertos                                        0
metrostotales                                          0
idzona                                              7179
piscina                                                0
anio                                                   0
mes                                                    0
dia                                                    0
cat_Apartamento                                        0
cat_Bodega comercial                                   0
cat_Casa                                               0
cat_Casa en condominio                                 0
cat_Casa uso de suelo                                  0
cat_Departamento Compartido                            0
cat_Duplex                     

In [59]:
df_train.isnull().sum()

antiguedad                                              0
habitaciones                                            0
garages                                                 0
banos                                                   0
metroscubiertos                                         0
metrostotales                                           0
idzona                                              28621
piscina                                                 0
precio                                                  0
anio                                                    0
mes                                                     0
dia                                                     0
cat_Apartamento                                         0
cat_Bodega comercial                                    0
cat_Casa                                                0
cat_Casa en condominio                                  0
cat_Casa uso de suelo                                   0
cat_Departamen

In [53]:
df_train.columns

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'piscina', 'precio', 'anio', 'mes', 'dia',
       'cat_Apartamento', 'cat_Bodega comercial', 'cat_Casa',
       'cat_Casa en condominio', 'cat_Casa uso de suelo',
       'cat_Departamento Compartido', 'cat_Duplex', 'cat_Edificio',
       'cat_Huerta', 'cat_Inmuebles productivos urbanos',
       'cat_Local Comercial', 'cat_Local en centro comercial', 'cat_Lote',
       'cat_Nave industrial', 'cat_Oficina comercial', 'cat_Otros',
       'cat_Quinta Vacacional', 'cat_Rancho', 'cat_Terreno',
       'cat_Terreno comercial', 'cat_Terreno industrial', 'cat_Villa',
       'ciudad_promedio_m2', 'idzona_promedio_m2', 'posicion_promedio_m2',
       'idzona_ordinal', 'ciudad_ordinal', 'provincia_ordinal',
       'suma_gimnasiousosmultiplespiscina',
       'suma_escuelascercanascentroscomercialescercanos', 'log_precio'],
      dtype='object')

In [54]:
df_test.columns

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'piscina', 'anio', 'mes', 'dia',
       'cat_Apartamento', 'cat_Bodega comercial', 'cat_Casa',
       'cat_Casa en condominio', 'cat_Casa uso de suelo',
       'cat_Departamento Compartido', 'cat_Duplex', 'cat_Edificio',
       'cat_Inmuebles productivos urbanos', 'cat_Local Comercial',
       'cat_Local en centro comercial', 'cat_Nave industrial',
       'cat_Oficina comercial', 'cat_Otros', 'cat_Quinta Vacacional',
       'cat_Rancho', 'cat_Terreno', 'cat_Terreno comercial',
       'cat_Terreno industrial', 'cat_Villa', 'cat_Huerta', 'cat_Lote',
       'ciudad_promedio_m2', 'idzona_promedio_m2', 'posicion_promedio_m2',
       'idzona_ordinal', 'ciudad_ordinal', 'provincia_ordinal',
       'suma_gimnasiousosmultiplespiscina',
       'suma_escuelascercanascentroscomercialescercanos'],
      dtype='object')

In [55]:
df_original = leer_csv('data/train.csv')

In [56]:
df_original.columns
df_original.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [57]:
df_modificado = df_original.copy()
df_modificado['lat'] = df_modificado['lat'].apply(lambda x: round(x,2))
df_modificado['posicion'] = df_modificado['lat'].map(str) + (df_modificado['lng'].map(str))
df_modificado


,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,posicion
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,NaN,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0,nannan
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,19.31,-99.227655,2013-06-28,0.0,0.0,0.0,1.0,1.0,3600000.0,19.31-99.2276548
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,NaN,NaN,2015-10-17,0.0,0.0,0.0,0.0,0.0,1200000.0,nannan
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,19.30,-99.688015,2012-03-09,0.0,0.0,0.0,1.0,1.0,650000.0,19.3-99.6880151
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,NaN,NaN,2016-06-07,0.0,0.0,0.0,0.0,0.0,1150000.0,nannan
5,126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,...,19.30,-99.148475,2014-03-18,0.0,0.0,0.0,0.0,1.0,1100000.0,19.3-99.14847480000005
6,139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,NaN,3.0,1.0,...,17.14,-96.803504,2016-02-23,0.0,0.0,0.0,0.0,0.0,1150000.0,17.14-96.8035043768
7,5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.0,4.0,2.0,...,20.67,-87.037968,2016-10-20,0.0,0.0,0.0,0.0,0.0,4200000.0,20.67-87.0379684
8,44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,...,NaN,NaN,2014-01-06,0.0,0.0,0.0,1.0,1.0,310000.0,nannan
9,134537,terreno,"terreno de 5.500m2 bardeado, uso de suelo h-20...",Terreno,Av. Morelos,Ixtapaluca,Edo. de México,NaN,NaN,NaN,...,19.32,-98.887000,2016-12-22,0.0,0.0,0.0,0.0,0.0,6200000.0,19.32-98.887
